In [1]:
import pandas as pd
import numpy as np
from pandas_schema import pandas_schema

In [2]:
columns = list(pandas_schema.keys())
datatypes = list(pandas_schema.values())
schema_list = list(zip(columns, datatypes))
#file_path = '/Users/matthewtryba/Desktop/subsampled_data.csv'
file_path = "Y:\\FannieMaeMortgageData\\subsampled_data_00125.csv"
sf_loan_performance = pd.read_csv(file_path, sep='|', header=None, names=columns, low_memory=False)

## ETL

In [3]:
def get_indices_by_type(schema_list):
    """
    Creates dictionary of set of datatypes in pandas_schema
    """
    indices_by_type = {datatype:[] for datatype in list(set(pandas_schema.values()))}

    for key,_ in indices_by_type.items():
        for i,_ in enumerate(schema_list):
            if key in schema_list[i][1]:
                indices_by_type[key].append(i)
    
    return indices_by_type


def process_dates(df, indices_by_type):
    """
    Converts date columns from MMYYYY to YYYY-MM-DD format, safely handling NaN values.
    :param df: DataFrame with data.
    :param indices_by_type: Dictionary with 'datetime64[ns]' key pointing to list of column indices.
    """
    error_col_indices = []

    for col in indices_by_type['datetime64[ns]']:
        try:
            # Direct conversion to string and zero-filling
            df.iloc[:, col] = df.iloc[:, col].astype(str).str.zfill(6)

            # Convert to datetime format, specifying the original format to speed up parsing
            df.iloc[:, col] = pd.to_datetime(df.iloc[:, col], format='%m%Y', errors='coerce')
        
        except ValueError:
            error_col_indices.append(col)
        
    # Optionally, log errors (if frequent):
    # if error_col_indices:
    #     print(f'Errors on columns  {error_col_indices}')
    
    return df


def preprocess_booleans(df, indices_by_type):
    """
    Converts columns with "Y" and "N" values to boolean. All other values are set to NULL.
    :param df: DataFrame with data.
    :param bool_columns: List of columns indices to be converted.
    """
    for col in indices_by_type['bool']:
        df.iloc[:,col] = df.iloc[:,col].apply(convert_to_bool)

    return df


def convert_to_bool(x):
    if x == 'N' or x == 'n':
        return False
    elif x == "Y" or x == 'y':
        return True
    else:
        return None    

In [4]:
indices_by_type = get_indices_by_type(schema_list)
process_dates(sf_loan_performance, indices_by_type)
preprocess_booleans(sf_loan_performance, indices_by_type)

# Create dictionary of non-numeric features to be used in .astype
non_numeric_schema = {}
for key, value in pandas_schema.items():
    if value not in ['float64', 'int64', 'Int64']:
        non_numeric_schema[key] = value

# Set schema for non-numeric features
sf_loan_performance.astype(non_numeric_schema)

,Reference_Pool_ID,Loan_Identifier,Monthly_Reporting_Period,Channel,Seller_Name,Servicer_Name,Master_Servicer,Original_Interest_Rate,Current_Interest_Rate,Original_UPB,...,ARM_Plan_Number,Borrower_Assistance_Plan,High_Loan_to_Value_HLTV_Refinance_Option_Indicator,Deal_Name,Repurchase_Make_Whole_Proceeds_Flag,Alternative_Delinquency_Resolution,Alternative_Delinquency_Resolution_Count,Total_Deferral_Amount,Payment_Deferral_Modification_Event_Indicator,Interest_Bearing_UPB
0,NaN,100891825768,2006-10-01,R,Other,Other,NaN,8.500,8.500,54000.0,...,NaN,NaN,False,NaN,False,NaN,NaN,NaN,7,NaN
1,NaN,100392032458,2001-08-01,R,"Citimortgage, Inc.",NaN,NaN,7.000,7.000,35000.0,...,NaN,NaN,False,NaN,False,NaN,NaN,NaN,7,NaN
2,NaN,100775189711,2001-08-01,R,Bishops Gate Residential Mortgage Trust,NaN,NaN,8.375,8.375,114000.0,...,NaN,NaN,False,NaN,False,NaN,NaN,NaN,7,NaN
3,NaN,100278675059,2000-04-01,C,"Bank Of America, N.A.",NaN,NaN,8.375,8.375,70000.0,...,NaN,NaN,False,NaN,False,NaN,NaN,NaN,7,NaN
4,NaN,100733360727,2002-02-01,C,"Jpmorgan Chase Bank, Na","Jpmorgan Chase Bank, Na",NaN,7.500,7.500,55000.0,...,NaN,NaN,False,NaN,False,NaN,NaN,NaN,7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3469958,NaN,136871031,2023-09-01,R,Other,Other,NaN,7.375,7.375,238000.0,...,NaN,7,False,NaN,False,7,NaN,NaN,7,NaN
3469959,NaN,136879797,2023-09-01,R,"Pulte Mortgage, L.L.C.",Other,NaN,6.375,6.375,207000.0,...,NaN,7,False,NaN,False,7,NaN,NaN,7,NaN
3469960,NaN,136880136,2023-09-01,C,"PennyMac Loan Services, LLC","PennyMac Loan Services, LLC",NaN,7.375,7.375,510000.0,...,NaN,7,False,NaN,False,7,NaN,NaN,7,NaN
3469961,NaN,136880566,2023-09-01,R,Other,"Specialized Loan Servicing, LLC",NaN,7.500,7.500,195000.0,...,NaN,7,False,NaN,False,7,NaN,NaN,7,NaN


In [18]:
# Drop columns with 100% missing values
columns_to_drop = []

for col in sf_loan_performance.columns:

    # Check if total na values is equal to length of dataframe
    if sf_loan_performance[col].isna().sum() == sf_loan_performance.shape[0]:
        columns_to_drop.append(col)

sf_loan_performance.drop(columns=columns_to_drop, inplace=True)

# EDA

In [19]:
sf_loan_performance.describe()

,Loan_Identifier,Original_Interest_Rate,Current_Interest_Rate,Original_UPB,Current_Actual_UPB,Original_Loan_Term,Loan_Age,Remaining_Months_to_Legal_Maturity,Remaining_Months_to_Maturity,Original_Loan_to_Value_Ratio_LTV,...,Credit_Enhancement_Proceeds,Repurchase_Make_Whole_Proceeds,Other_Foreclosure_Proceeds,Modification_Related_Non_Interest_Bearing_UPB,Principal_Forgiveness_Amount,Mortgage_Insurance_Type,Foreclosure_Principal_Write_off_Amount,Alternative_Delinquency_Resolution_Count,Total_Deferral_Amount,Payment_Deferral_Modification_Event_Indicator
count,3.469963e+06,3.469963e+06,3.422378e+06,3.469963e+06,3.469963e+06,3.469963e+06,3.422354e+06,3.422168e+06,3.369957e+06,3.469963e+06,...,574.000000,499.000000,653.000000,50843.000000,22147.0,631185.000000,22161.000000,12713.000000,12530.000000,3469963.0
mean,4.827340e+11,4.836708e+00,4.803512e+00,1.908058e+05,1.510166e+05,3.046243e+02,4.441576e+01,2.622492e+02,2.534742e+02,6.959998e+01,...,16017.357247,7996.858016,4605.034916,7249.331640,0.0,1.065054,16.536959,1.048533,12374.312808,7.0
std,3.028286e+11,1.284084e+00,1.285489e+00,1.148197e+05,1.182092e+05,8.312776e+01,4.098809e+01,9.534768e+01,9.797084e+01,1.775971e+01,...,26703.877951,31350.126371,16611.643428,22004.246295,0.0,0.248286,733.777635,0.233158,11674.682909,0.0
min,9.747312e+07,1.500000e+00,1.500000e+00,5.000000e+03,0.000000e+00,6.000000e+01,-1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,89.780000,7.0
25%,2.309285e+11,3.810000e+00,3.750000e+00,1.050000e+05,6.725898e+04,1.800000e+02,1.300000e+01,1.720000e+02,1.670000e+02,5.900000e+01,...,0.000000,0.000000,3.550000,0.000000,0.0,1.000000,0.000000,1.000000,4345.495000,7.0
50%,4.872230e+11,4.826000e+00,4.750000e+00,1.630000e+05,1.270101e+05,3.600000e+02,3.200000e+01,3.030000e+02,2.940000e+02,7.500000e+01,...,0.000000,0.000000,632.310000,0.000000,0.0,1.000000,0.000000,1.000000,8291.920000,7.0
75%,7.436405e+11,5.875000e+00,5.750000e+00,2.500000e+05,2.124021e+05,3.600000e+02,6.500000e+01,3.410000e+02,3.380000e+02,8.000000e+01,...,25245.940000,0.000000,2732.500000,0.000000,0.0,1.000000,0.000000,1.000000,16526.747500,7.0
max,9.999999e+11,1.150000e+01,1.150000e+01,1.562000e+06,1.537338e+06,3.600000e+02,2.930000e+02,4.810000e+02,4.800000e+02,9.700000e+01,...,158077.850000,362476.520000,187746.600000,338000.000000,0.0,3.000000,82093.330000,5.000000,101706.300000,7.0


In [24]:
duplicated_rows = sf_loan_performance.duplicated()
print(duplicated_rows.sum())

0


0
